In [103]:
import numpy as np
np.random.seed(5)

import functions_class as fx
import classx as cl
import log_reg_functions as lrf
import loaddata as ld


import time
import matplotlib.pyplot as plt
import pandas as pd
import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import log_loss, f1_score


# A ) 
Load data

In [104]:
x,y = ld.load_data(scaler='minmax')

xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size=0.25, random_state= 0, shuffle=True, stratify=y)

xtrain,xval,ytrain,yval = train_test_split(xtrain,ytrain, test_size=0.25, random_state= 0, shuffle=True, stratify=ytrain)
# do this because want our test to be seperate, first splitting into test and train, and then splitting traindata 
#into training and validation

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape



### f1-score is better when splitting data 50/50, this indicates that our algorithm learns more. The accuracy is expected to be lower for test data in this case. The dataset is biased, (read more) and therefore will give higher accuracy on biased datasets. 

In [107]:

indices = np.where(y == 1)
indices_zero = np.where(y == 0)
datapoints = np.random.choice(indices_zero[0], size=y[indices[0]].shape[0], replace=False)

x_new = np.vstack((x[indices[0],:],x[datapoints,:]))
y_new = np.vstack((y[indices[0]],y[datapoints]))

xtrain = x_new
ytrain = y_new

nx_train, ny_train = xtrain.shape
nx_test, ny_test = xtest.shape


# B) 
Egen logistic regression med gradient descent.


In [108]:
eta = 0.1 # learning rate
doplot = True
doprint = False
Niteration = 250
beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_val=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

start = time.time()
for iter in range(Niteration):
    
    sig = lrf.sigmoid(xtrain@beta)
    gradient = lrf.gradient_ols(xtrain,ytrain,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(xtrain@beta,ytrain.T)
    cost_val = lrf.cost_log_ols(xval@beta,yval.T) # do this for testdata at the end. 
    #Log Loss function from sklearn
    logloss=log_loss(ytrain, np.round(xtrain@beta), eps=1e-16, normalize=True)
    if doprint:
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_val)
    if doplot:
        costvec.append(cost.ravel())
        costvec_val.append(cost_val.ravel())
        loglvec.append(logloss)
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_val, 'r')
        #plt.plot(xaxis, loglvec, 'g')
        plt.pause(1e-12)
plt.show()    
end = time.time()
print(end - start)
   
# et relevant plot er iterativt plot. 

18.107311010360718


In [110]:
# making confusion matrix to check observed data with model predictions. 

predictions = xtest@beta
sig_val = lrf.sigmoid(predictions)
sig_val = np.round(sig_val)

cm = confusion_matrix(ytest , sig_val.astype(int))
print(cm)

[[4272 1569]
 [ 773  886]]


# Accuracy. 
Både egen kode og tester med scikit. 

In [111]:

activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')
print(f1_score(ytrain, classes))

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')
print(f1_score(yval, classes))


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')
print(f1_score(ytest, classes))



64.04460518384569 % Training Accuracy
0.6024658447184272
69.04888888888888 % Validation Accuracy
0.4349237260629666
68.77333333333333 % Test Accuracy
0.4307243558580457


# Egen logistisk regresjon med stokastisk gradient descent

In [112]:
eta = 0.1 # learning rate
doplot = True
doprint = False
Niteration = 250
batch_size = 3000
batch_size_held_out = nx_train-batch_size

beta = np.random.randn(x.shape[1],1)
costvec=[]
costvec_test=[]
loglvec=[]
xaxis=[]

%matplotlib qt
#plt.axis([0, Niteration, 0, 13])

indexes = np.arange(nx_train)


start = time.time()
for iter in range(Niteration):
    datapoints = np.random.choice(indexes, size=batch_size, replace=False)
    batch_x = xtrain[datapoints,:]
    batch_y = ytrain[datapoints]
    
    batch_x_held_out = np.delete(xtrain, datapoints, axis=0)
    batch_y_held_out = np.delete(ytrain, datapoints).reshape([batch_size_held_out,1])
    
    sig = lrf.sigmoid(batch_x@beta)
    gradient = lrf.gradient_ols(batch_x,batch_y,sig)
    beta -= eta*gradient
    
    #Cost function
    cost = lrf.cost_log_ols(batch_x@beta,batch_y.T)
    cost_test = lrf.cost_log_ols(batch_x_held_out@beta,batch_y_held_out.T)
    #Log Loss function from sklearn
    if doprint:
        logloss=log_loss(batch_y, np.round(batch_x@beta), eps=1e-16, normalize=True)
        print('Cost', cost,'&','Log loss', logloss,'&','Cost test', cost_test)
    if doplot:
        costvec.append(cost.ravel())
        costvec_test.append(cost_test.ravel())
        xaxis.append(iter+1)
        plt.plot(xaxis, costvec, 'b')
        plt.plot(xaxis, costvec_test, 'r')
        plt.pause(1e-12)
plt.show()

end = time.time()
print(end - start)
 

27.082622289657593


In [113]:
# making confusion matrix to check observed data with model predictions. 

predictions = xtest@beta
sig_val = lrf.sigmoid(predictions)
sig_val = np.round(sig_val)

cm = confusion_matrix(ytest , sig_val.astype(int))
print(cm)

[[4358 1483]
 [ 745  914]]


In [114]:
activation =lrf.sigmoid(xtrain@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytrain)/len(activation),'% Training Accuracy')
print(f1_score(ytrain, classes))

activation =lrf.sigmoid(xval@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==yval)/len(activation),'% Validation Accuracy')
print(f1_score(yval, classes))


activation =lrf.sigmoid(xtest@beta) 
classes = np.zeros([len(activation)])
classes=np.round(activation)
print(100*np.sum(classes==ytest)/len(activation),'% Test Accuracy')
print(f1_score(ytest, classes))


64.7302591922845 % Training Accuracy
0.6101440826184725
70.54222222222222 % Validation Accuracy
0.454395785314455
70.29333333333334 % Test Accuracy
0.4506903353057199


In [12]:

model = LogisticRegression()
model.fit(xtrain, ytrain)
predicted_classes = model.predict(xtrain)
accuracy = accuracy_score(ytrain.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytrain, predicted_classes)

print(accuracy, '% Training Accuracy')

predicted_classes = model.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),predicted_classes)
accuracy = accuracy * 100
parameters = model.coef_
log_loss(ytest, predicted_classes)

print(accuracy, '% Test Accuracy')

C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


71.26280892103676 % Training Accuracy
77.36 % Test Accuracy


In [ ]:
from sklearn.preprocessing import OneHotEncoder


onehotencoder = OneHotEncoder(categories="auto", sparse = False)
X = ColumnTransformer(
    [("", onehotencoder, [1,2,3,5,6,7,8,9,10]),],
    remainder="passthrough"
).fit_transform(X)

# C) 
Neural Network. 

In [112]:
#self.X_data_full = X_data 
#self.Y_data_full = Y_data

n_inputs, n_features = x.shape

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
eta = 0.001
lmbd = 10e-05


n_hidden_neurons=50  # velge andre verdier om vi vil
n_categories=2
epochs=100
batch_size=80
eta=0.01
n_hidden_neurons2=20  # velge andre verdier om vi vil



hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_weights2 = np.random.randn(n_hidden_neurons, n_hidden_neurons2)
hidden_bias = np.zeros(n_hidden_neurons) + 0.1
hidden_bias2 = np.zeros(n_hidden_neurons2) + 0.1

output_weights = np.random.randn(n_hidden_neurons2, n_categories)
output_bias = np.zeros(n_categories) + 0.1

In [113]:
print(hidden_weights.shape)
print(hidden_bias.shape)
print(hidden_weights2.shape)
print(hidden_bias2.shape)
print(output_weights.shape)
print(output_bias.shape)
print(x.shape)

(91, 50)
(50,)
(50, 20)
(20,)
(20, 2)
(2,)
(30000, 91)


In [114]:
z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
a_h2 = lrf.sigmoid(z_h2)

z_o = np.matmul(a_h2, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [115]:
print(x.shape)
print(z_h.shape)
print(a_h.shape)
print(z_h2.shape)
print(a_h2.shape)
print(z_o.shape)
print(probabilities)

(30000, 91)
(30000, 50)
(30000, 50)
(30000, 20)
(30000, 20)
(30000, 2)
[[0.86973101 0.13026899]
 [0.18130462 0.81869538]
 [0.16498562 0.83501438]
 ...
 [0.94564342 0.05435658]
 [0.03933096 0.96066904]
 [0.38076984 0.61923016]]


In [120]:
error_output = probabilities - y
error_hidden2 = np.matmul(error_output, output_weights.T) * a_h2 * (1 - a_h2)
error_hidden = np.matmul(error_hidden2, hidden_weights2.T) * a_h * (1 - a_h) #a_h * (1 - a_h) is specific for sigmoid


output_weights_gradient = np.matmul(a_h2.T, error_output)
output_bias_gradient = np.sum(error_output, axis=0)

hidden_weights_gradient2 = np.matmul(a_h.T, error_hidden2)
hidden_bias_gradient2 = np.sum(error_hidden2, axis=0)

hidden_weights_gradient = np.matmul(x.T, error_hidden)
hidden_bias_gradient = np.sum(error_hidden, axis=0)

if lmbd > 0.0:
    output_weights_gradient += lmbd * output_weights
    hidden_weights_gradient2 += lmbd * hidden_weights2
    hidden_weights_gradient += lmbd * hidden_weights
    
output_weights -= eta * output_weights_gradient
output_bias -= eta * output_bias_gradient

hidden_weights2 -= eta * hidden_weights_gradient2
hidden_bias2 -= eta * hidden_bias_gradient2

hidden_weights -= eta * hidden_weights_gradient
hidden_bias -= eta * hidden_bias_gradient


z_h = np.matmul(x, hidden_weights) + hidden_bias
a_h = lrf.sigmoid(z_h)

z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
a_h2 = lrf.sigmoid(z_h2)

z_o = np.matmul(a_h2, output_weights) + output_bias

exp_term = np.exp(z_o)
probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)


C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


In [28]:
from sklearn.preprocessing import OneHotEncoder
x=xtrain
y=ytrain

onehotencoder = OneHotEncoder(categories="auto", sparse = False)
y = ColumnTransformer(
    [("", onehotencoder, [0]),],
    remainder="passthrough"
).fit_transform(y)
print(y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [29]:
def test_nan(a):
    print(np.isnan(a).sum())
    return

In [3]:
Niter = 200


n_inputs, n_features = x.shape

#self.epochs = epochs
#self.batch_size = batch_size
#self.iterations = self.n_inputs // self.batch_size
#lmbd = 0.1


n_hidden_neurons=50  # velge andre verdier om vi vil
n_categories=2

eta=0.0001
n_hidden_neurons2=20  # velge andre verdier om vi vil

bias=0

hidden_weights = np.random.randn(n_features, n_hidden_neurons)
hidden_weights2 = np.random.randn(n_hidden_neurons, n_hidden_neurons2)
hidden_bias = np.zeros(n_hidden_neurons) + bias
hidden_bias2 = np.zeros(n_hidden_neurons2) + bias

output_weights = np.random.randn(n_hidden_neurons2, n_categories)
output_bias = np.zeros(n_categories) + bias

start = time.time()
for i in range(Niter):
    z_h = np.matmul(x, hidden_weights) + hidden_bias
    #test_nan(z_h)
    a_h = lrf.sigmoid(z_h)
    #test_nan(a_h)
    z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
    #test_nan(z_h2)
    a_h2 = lrf.sigmoid(z_h2)
    #test_nan(a_h2)
    z_o = np.matmul(a_h2, output_weights) + output_bias
    
    #print(np.min(z_o), np.max(z_o))
    #probabilities = lrf.sigmoid(z_o)
    exp_term = np.exp(z_o)
    #test_nan(exp_term)
    #print(np.unique(exp_term))
    probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)
    #test_nan(probabilities)
    
    error_output = probabilities - y 
    #print((1-error_output).sum()/len(error_output))
    #print(np.isnan(probabilities).sum())
    error_hidden2 = np.matmul(error_output, output_weights.T) * a_h2 * (1 - a_h2)
    error_hidden = np.matmul(error_hidden2, hidden_weights2.T) * a_h * (1 - a_h) #a_h * (1 - a_h) is specific for sigmoid

    output_weights_gradient = np.matmul(a_h2.T, error_output)
    output_bias_gradient = np.sum(error_output, axis=0)

    hidden_weights_gradient2 = np.matmul(a_h.T, error_hidden2)
    hidden_bias_gradient2 = np.sum(error_hidden2, axis=0)

    hidden_weights_gradient = np.matmul(x.T, error_hidden)
    hidden_bias_gradient = np.sum(error_hidden, axis=0)
    
    #if lmbd > 0.0:
     #   output_weights_gradient += lmbd * output_weights
      #  hidden_weights_gradient2 += lmbd * hidden_weights2
       # hidden_weights_gradient += lmbd * hidden_weights
    
    #test_nan(output_weights_gradient)
    #test_nan(output_weights_bias)
    
    output_weights -= eta * output_weights_gradient
    output_bias -= eta * output_bias_gradient
    #print(output_weights_gradient)
    
    
    hidden_weights2 -= eta * hidden_weights_gradient2
    hidden_bias2 -= eta * hidden_bias_gradient2

    hidden_weights -= eta * hidden_weights_gradient
    hidden_bias -= eta * hidden_bias_gradient

end = time.time()
print(end - start)



C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide


23.477296352386475


In [46]:
#def forward_pass():
for i in range(Niter):
    z_h = np.matmul(xtest, hidden_weights) + hidden_bias
    a_h = lrf.sigmoid(z_h)

    z_h2 = np.matmul(a_h, hidden_weights2) + hidden_bias2
    a_h2 = lrf.sigmoid(z_h2)

    z_o = np.matmul(a_h2, output_weights) + output_bias
    exp_term = np.exp(z_o)
    probabilities = exp_term / np.sum(exp_term, axis=1, keepdims=True)

In [47]:
print(probabilities)

[[0.89081651 0.10918349]
 [0.91380032 0.08619968]
 [0.90385102 0.09614898]
 ...
 [0.89687279 0.10312721]
 [0.88505932 0.11494068]
 [0.93537178 0.06462822]]


In [49]:

classes=np.argmax(probabilities,axis=1)
#y=np.argmax(y,axis=1)
print(classes)
print(100*np.sum(classes.T==ytest.ravel())/len(ytest.ravel()),'% Training Accuracy')
print(f1_score(ytest, classes))

cm = confusion_matrix(ytest , classes)
print(cm)


[0 0 0 ... 0 0 0]
81.8 % Training Accuracy
0.4678362573099415
[[5535  306]
 [1059  600]]


In [27]:
np.unique(classes)

array([0], dtype=int64)

In [4]:

class ANN():
    def __init__(self, lmb=0, bias=0, eta=0.0001):
        self.lmb=lmb
        self.bias = bias
        self.layers=dict()
        self.n_layers=int()
        
        self.pred=dict()
        self.act=dict()
        self.eta=eta
        
    def add_layers(self, n_neurons=[20,5], n_features=[20,5], n_layers=2):
        self.n_layers=n_layers
        for i in range(n_layers):
            layer_weights = np.random.randn(n_features[i], n_neurons[i])
            self.layers['w'+str(i)] = layer_weights
            layer_bias = np.zeros(n_neurons[i]) + self.bias
            self.layers['b'+str(i)] = layer_bias
    
    def feed(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
  
    def back(self, design, data):
        
        for i in np.arange(self.n_layers-1,0,-1):
            if i==self.n_layers-1:
                error = self.act[str(i)] - data
            else:
                error = np.matmul(error, self.layers['w'+str(i+1)].T) * self.act[str(i)] * (1 - self.act[str(i)])
            if i == 0:
                gradients_weights = np.matmul(design.T, error)
                gradients_bias = np.sum(error, axis=0)
            else:
                gradients_weights = np.matmul(self.act[str(i-1)].T, error)
                gradients_bias = np.sum(error, axis=0)
            
            
            self.layers['w'+str(i)] -= self.eta * gradients_weights
            self.layers['b'+str(i)] -= self.eta * gradients_bias
   
    def feed_out(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
        return self.act

In [12]:
neural_net = ANN(lmb=0.3, bias=0.24, eta=0.0001)            
neural_net.add_layers(n_features=[91,50,20], n_neurons = [50,20,1] , n_layers=3)

x=xtrain
y=ytrain

start = time.time()
for i in range(3000):
    neural_net.feed(x)
    neural_net.back(x,y)

end = time.time()
print(end - start)



152.62720441818237


In [31]:
activ=neural_net.feed_out(x)['2']
print(activ)

classes=np.round(activ)
print(classes)

print(100*np.sum(classes==y)/len(activation),'% Training Accuracy')
#print(f1_score(ytrain, classes)) 

cm = confusion_matrix(y , classes)
print(cm)

[[0.00178645]
 [0.00115319]
 [0.00128447]
 ...
 [0.00133755]
 [0.42446725]
 [0.00115894]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
78.30518518518518 % Training Accuracy
[[13121    21]
 [ 3640    93]]


In [89]:
class ANN():
    def __init__(self, lmb=0, bias=0, eta=0.0001):
        self.lmb=lmb
        self.bias = bias
        self.layers=dict()
        self.n_layers=int()
        
        self.pred=dict()
        self.act=dict()
        self.eta=eta
        
    def add_layers(self, n_neurons=[20,5], n_features=[20,5], n_layers=2):
        self.n_layers=n_layers
        for i in range(n_layers):
            layer_weights = np.random.randn(n_features[i], n_neurons[i])
            self.layers['w'+str(i)] = layer_weights
            layer_bias = np.zeros(n_neurons[i]) + self.bias
            self.layers['b'+str(i)] = layer_bias
    
    def feed(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
  
    def back(self, design, data, derivative=[lrf.sigmoid_deriv, lrf.sigmoid_deriv, lrf.sigmoid_deriv]):
        
        for i in np.arange(self.n_layers-1,0,-1):
            if i==self.n_layers-1:
                error = self.act[str(i)] - data
            else:
                error = np.matmul(error, self.layers['w'+str(i+1)].T) * derivative[i](self.act[str(i)])
            print(error.shape)
            if i == 0:
                gradients_weights = np.matmul(design.T, error)
                gradients_bias = np.sum(error, axis=0)
            else:
                gradients_weights = np.matmul(self.act[str(i-1)].T, error)
                gradients_bias = np.sum(error, axis=0)
            
            
            self.layers['w'+str(i)] -= self.eta * gradients_weights
            self.layers['b'+str(i)] -= self.eta * gradients_bias
   
    def feed_out(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
        return self.act
    
    def train(self, epochs, batch_size, x, y, activation, derivative):
        tmp=int(len(y)/batch_size)
        Niter = min(200,tmp)
        indexes = np.arange(len(y))
        for i in range(epochs):
            for j in range(Niter):
                datapoints = np.random.choice(indexes, size=batch_size, replace=False)
                batch_x = x[datapoints,:]
                batch_y = y[datapoints]
                
                self.feed(batch_x, activation)
                self.back(batch_x,batch_y, derivative)
            pred = self.feed_out(x, activation)[str(self.n_layers-1)]
            cost = lrf.cost_log_ols(pred,y.T)
            print('Epoch', i, 'loss', cost )
        

In [117]:
class ANN():
    def __init__(self, lmb=0, bias=0, eta=0.0001, mode = 'classification'):
        self.lmb=lmb
        self.bias = bias
        self.layers=dict()
        self.n_layers=int()
        
        self.pred=dict()
        self.act=dict()
        self.eta=eta
        self.mode = mode
        
    def add_layers(self, n_neurons=[20,5], n_features=[20,5], n_layers=2):
        self.n_layers=n_layers
        for i in range(n_layers):
            layer_weights = np.random.randn(n_features[i], n_neurons[i])
            self.layers['w'+str(i)] = layer_weights
            layer_bias = np.zeros(n_neurons[i]) + self.bias
            self.layers['b'+str(i)] = layer_bias
    
    def feed(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
  
    def back(self, design, data, derivative=[lrf.sigmoid_deriv, lrf.sigmoid_deriv, lrf.sigmoid_deriv]):
        
        for i in np.arange(self.n_layers-1,0,-1):
            if i==self.n_layers-1:
                error = self.act[str(i)] - data
            else:
                error = np.matmul(error, self.layers['w'+str(i+1)].T) * derivative[i](self.act[str(i)])
            if i == 0:
                gradients_weights = (np.matmul(design.T, error))/len(data)
                gradients_bias = (np.sum(error, axis=0))/len(data)
            else:
                gradients_weights = (np.matmul(self.act[str(i-1)].T, error))/len(data)
                gradients_bias = (np.sum(error, axis=0))/len(data)
            
            if self.lmb>0.0:
                gradients_weights += self.lmb * self.layers['w'+str(i)]
            
            self.layers['w'+str(i)] -= self.eta * gradients_weights
            self.layers['b'+str(i)] -= self.eta * gradients_bias
   
    def feed_out(self, design, activation=[lrf.sigmoid,lrf.sigmoid, lrf.sigmoid]):
        
        for i in range(self.n_layers):
            
            if i==0:
                self.pred[str(i)] = np.matmul(design, self.layers['w'+str(i)]) + self.layers['b'+str(i)]
                self.act[str(i)] = activation[i](self.pred[str(i)])
            else:
                self.pred[str(i)] = np.matmul(self.act[str(i-1)], self.layers['w'+str(i)]) + self.layers['b'+str(i)]                
                self.act[str(i)] = activation[i](self.pred[str(i)])
        return self.act
    
    def train(self, epochs, batch_size, x, y, activation, derivative, verbose=False):
        tmp=int(len(y)/batch_size)
        Niter = min(200,tmp)
        indexes = np.arange(len(y))
        for i in range(epochs):
            for j in range(Niter):
                datapoints = np.random.choice(indexes, size=batch_size, replace=False)
                batch_x = x[datapoints,:]
                batch_y = y[datapoints]
                
                self.feed(batch_x, activation)
                self.back(batch_x,batch_y, derivative)
            if verbose:
                if self.mode == 'regression':
                    pred = self.feed_out(x, activation)[str(self.n_layers-1)]
                    cost = fx.MSE(pred.ravel(),y.ravel())
                    print('Epoch', i, 'loss', cost )
                if self.mode == 'classification':
                    pred = self.feed_out(x, activation)[str(self.n_layers-1)]
                    cost = lrf.cost_log_ols(pred,y.T)
                    print('Epoch', i, 'loss', cost )


In [122]:
neural_net = ANN(lmb=0.0001, bias=0, eta=0.1, mode = 'classification')            
neural_net.add_layers(n_features=[91,50,50,20], n_neurons = [50,50,20,1] , n_layers=4)

x=xtrain
y=ytrain

activation = [lrf.relu, lrf.relu, lrf.sigmoid, lrf.sigmoid]
derivative = [lrf.relu_deriv, lrf.relu_deriv, lrf.sigmoid_deriv, lrf.sigmoid_deriv]

start = time.time()


epochs=100
batch_size=2500
neural_net.train(epochs, batch_size, x,y,activation,derivative , verbose=True)

end = time.time()
print(end - start)


Epoch 0 loss [[0.7353072]]
Epoch 1 loss [[0.72363795]]
Epoch 2 loss [[0.7163811]]
Epoch 3 loss [[0.71200495]]
Epoch 4 loss [[0.70860239]]
Epoch 5 loss [[0.70579477]]
Epoch 6 loss [[0.70402728]]
Epoch 7 loss [[0.7026137]]
Epoch 8 loss [[0.70063961]]
Epoch 9 loss [[0.69938504]]
Epoch 10 loss [[0.69843749]]
Epoch 11 loss [[0.69696573]]
Epoch 12 loss [[0.69609868]]
Epoch 13 loss [[0.69528406]]
Epoch 14 loss [[0.69342501]]
Epoch 15 loss [[0.69329503]]
Epoch 16 loss [[0.69350476]]
Epoch 17 loss [[0.6926193]]
Epoch 18 loss [[0.69277523]]
Epoch 19 loss [[0.69163142]]
Epoch 20 loss [[0.69097033]]
Epoch 21 loss [[0.69144856]]
Epoch 22 loss [[0.69034087]]
Epoch 23 loss [[0.69048756]]
Epoch 24 loss [[0.68967675]]
Epoch 25 loss [[0.6898435]]
Epoch 26 loss [[0.68932181]]
Epoch 27 loss [[0.68809918]]
Epoch 28 loss [[0.68859842]]
Epoch 29 loss [[0.68849373]]
Epoch 30 loss [[0.68874404]]
Epoch 31 loss [[0.68745494]]
Epoch 32 loss [[0.68744678]]
Epoch 33 loss [[0.68706213]]
Epoch 34 loss [[0.68845442]]


In [123]:
activ=neural_net.feed_out(x, activation)['3']
print(activ)

classes=np.round(activ)
print(classes)

print(100*np.sum(classes==y)/len(activ),'% Training Accuracy')
#print(f1_score(ytrain, classes)) 

cm = confusion_matrix(y , classes)
print(cm)

[[0.73577532]
 [0.4834637 ]
 [0.79685989]
 ...
 [0.34592695]
 [0.52190359]
 [0.3434502 ]]
[[1.]
 [0.]
 [1.]
 ...
 [0.]
 [1.]
 [0.]]
67.76672694394213 % Training Accuracy
[[5118 1518]
 [2760 3876]]


In [124]:
activ=neural_net.feed_out(xtest, activation)['3']
print(activ)

classes=np.round(activ)
print(classes)

print(100*np.sum(classes==ytest)/len(activ),'% T Accuracy')
#print(f1_score(ytrain, classes)) 

cm = confusion_matrix(ytest , classes)
print(cm)

[[0.31821408]
 [0.20298325]
 [0.37412292]
 ...
 [0.46770299]
 [0.31596628]
 [0.14478096]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
72.84 % T Accuracy
[[4487 1354]
 [ 683  976]]


In [76]:
from sklearn.neural_network import MLPClassifier


mlp = MLPClassifier(hidden_layer_sizes=(50,50,20,), max_iter=200, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

mlp.fit(x, y.ravel())
print("Training set score: %f" % mlp.score(x, y))
print("Test set score: %f" % mlp.score(xtest, ytest))



Iteration 1, loss = 0.47058199
Iteration 2, loss = 0.43925224
Iteration 3, loss = 0.43579302
Iteration 4, loss = 0.43459617
Iteration 5, loss = 0.43174006
Iteration 6, loss = 0.43196502
Iteration 7, loss = 0.43020106
Iteration 8, loss = 0.43028145
Iteration 9, loss = 0.42806578
Iteration 10, loss = 0.42829283
Iteration 11, loss = 0.42715236
Iteration 12, loss = 0.42587892
Iteration 13, loss = 0.42510128
Iteration 14, loss = 0.42576809
Iteration 15, loss = 0.42460461
Iteration 16, loss = 0.42338827
Iteration 17, loss = 0.42229345
Iteration 18, loss = 0.42162485
Iteration 19, loss = 0.41993946
Iteration 20, loss = 0.42177044
Iteration 21, loss = 0.42035700
Iteration 22, loss = 0.41894649
Iteration 23, loss = 0.41750074
Iteration 24, loss = 0.41702136
Iteration 25, loss = 0.41689325
Iteration 26, loss = 0.41679972
Iteration 27, loss = 0.41538672
Iteration 28, loss = 0.41519998
Iteration 29, loss = 0.41547898
Iteration 30, loss = 0.41315813
Iteration 31, loss = 0.41298859
Iteration 32, los

C:\Users\vemundst\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [77]:
pred = mlp.predict(x)
accuracy = accuracy_score(y.flatten(),pred)
accuracy = accuracy * 100
print(accuracy)

cm = confusion_matrix(y , pred)
print(cm)

87.78666666666666
[[12813   329]
 [ 1732  2001]]


In [78]:
pred = mlp.predict(xtest)
accuracy = accuracy_score(ytest.flatten(),pred)
accuracy = accuracy * 100
print(accuracy)

cm = confusion_matrix(ytest , pred)
print(cm)

79.33333333333333
[[5424  417]
 [1133  526]]
